In [1]:
import psycopg2
import json
import io
import csv
from dateutil.parser import isoparse

In [2]:
from sql_queries import staging_drop_table_queries, staging_create_table_queries

In [3]:
# Connect to local database

connection = psycopg2.connect(
    host="localhost",
    port=6021,
    database="eksi",
    user="postgres",
    password="docker")

cursor = connection.cursor()

In [4]:
# Drop and create staging tables

for query in staging_drop_table_queries:
    cursor.execute(query)
    connection.commit()

for query in staging_create_table_queries:
    cursor.execute(query)
    connection.commit()

In [5]:
with open('../../data/eksi_preprocessed/db_author_detail.json', 'r', encoding='utf-8') as jsonfile:
    line_count = 0
    for item in jsonfile:
        line_count += 1
        author = json.loads(item)
        try:
            tab_splited_values = io.StringIO("\t".join(map(str,author.values())))
            cursor.copy_from(tab_splited_values, 'staging_author_details', sep='\t', null="None")
            connection.commit()
        except (Exception, psycopg2.DatabaseError) as e:
            print(e)
            break
    print(line_count)

162728


In [11]:
try:
    query = "select count(*) from staging_author_details"
    cursor.execute(query)
    result = cursor.fetchone()
    print(result)
except (Exception, psycopg2.DatabaseError) as e:
    print(e)

(162728,)


In [11]:
with open('../../data/eksi_preprocessed/db_topics.json', 'r', encoding='utf-8') as jsonfile:
    tab_splited_values = ""
    for index, item in enumerate(jsonfile):
        if index%500000 == 0:
            print(index)
        topic = json.loads(item)
        try:
            tab_splited_values += "\t".join(map(str,topic.values())) + "\n"
        except (Exception, psycopg2.DatabaseError) as e:
            print(e)
            break

0
500000
1000000
1500000


In [13]:
tab_splited_values.count('\n')

1890596

In [14]:
try:
    tab_splited_values_io = io.StringIO(tab_splited_values)
    cursor.copy_from(tab_splited_values_io, 'staging_topics', sep='\t', null="None")
    connection.commit()
except (Exception, psycopg2.DatabaseError) as e:
    print(e)

In [15]:
try:
    query = "select count(*) from staging_topics"
    cursor.execute(query)
    result = cursor.fetchone()
    print(result)
except (Exception, psycopg2.DatabaseError) as e:
    print(e)

(1890596,)


In [35]:
with open('../../data/eksi_preprocessed/db_entries.json', 'r', encoding='utf-8') as jsonfile:
    line_count = 0
    for item in jsonfile:
        line_count += 1
        entry = json.loads(item)
        try:
            tab_splited_values = io.StringIO("\t".join(map(str,entry.values())))
            cursor.copy_from(tab_splited_values, 'staging_entries', sep='\t', null="None")
            connection.commit()
        except (Exception, psycopg2.DatabaseError) as e:
            print(entry)
            print(e)
            break
    print(line_count)

1000


In [36]:
try:
    query = "select count(*) from staging_entries"
    cursor.execute(query)
    result = cursor.fetchone()
    print(result)
except (Exception, psycopg2.DatabaseError) as e:
    print(e)

(1000,)


In [41]:
# cursor.execute("ROLLBACK")
# connection.commit()

In [4]:
from sql_queries import drop_table_queries, create_table_queries,insert_table_queries

In [5]:
# Drop and create staging tables

for query in drop_table_queries:
    cursor.execute(query)
    connection.commit()

for query in create_table_queries:
    cursor.execute(query)
    connection.commit()

In [6]:
for query in insert_table_queries:
    cursor.execute(query)
    connection.commit()

In [7]:
from sql_queries import entry_table_insert

In [8]:
cursor.execute(entry_table_insert)
connection.commit()